## N Queens Placement
#### Find a solution for placing N Queens on a  NxN such that no two queens attack each other

In [2]:
import itertools

## Brute force search using permutations

Board is represented as 8x8 matrix. (0,0) corresponds to top left corner.
Model the solution as a list of 8 ints. Each entry correpsonds to row number where the queen is placed.
eg. [1, 0, ... ] = Queen in 2nd row (we are using zero index, hence 2nd row = 1), Queen in first row, ....

Each possible solution is a permutation of digits [0, 1, ... N-1] with no replacement.  A valid solution ensures that no two positions are diagnonal. i.e. horiz distance != vertical distance.  

We will start generating all possible solutions and check each one for validity. If it passes diag validity check, we find our solution.

In [3]:
%%time

def is_solution(solution): 
    '''
    Check a specific solution permutation is valid. Solution should be list of N ints, corresponding
    to row positions of the queen for each column.
    '''
    # generate all combinations pairs of x locations
    for (x1, x2) in itertools.combinations(range(len(solution)), 2):
        y1, y2 = solution[x1], solution[x2]
        # check if diag (horiz distance == vertical distance)
        if abs(x2-x1) == abs(y2-y1):
            return False        
    # its a valid solution
    return True

N = 8
for solution in itertools.permutations(range(N)):
    if is_solution(solution):
        print(solution)
        break


(0, 4, 7, 5, 2, 6, 1, 3)
Wall time: 9.5 ms


## Search with Backtracking

This approach models the board as 8x8 array, and tryies to find a solution by placing each queen at the next possible position allowed position. Board updated after each placement (i.e. all new cells attacked by last placement are marked as Blocked, to prevent future placments).

If we are able to place all N queens, we have a solution. If we are do not find a open position to place a queen at any given step, we remove the queen from last placement and move it to next possible position on the row.

We accomplish backtracking by making a new board for each step. If we fail to reach a solution, we discard the board and start with next position.

In [6]:
import enum
import copy


class POS(enum.Enum):
    '''
    Value placed in each cell. 
    '''
    Empty = " "
    Block = "X"
    Place = "Q" 
    
    def __repr__(self):
        return self.value
    
class Board:
    '''
    Board represented as NxN (list of lists)
    '''
    def __init__(self, n, initialize=True):        
        self._board = None
        if initialize:
            self._board = [[POS.Empty] * n for i in range(n)]
        
    def copy(self): 
        '''
        Return a deep copy of new board.
        '''
        new_board = Board(0, initialize=False)
        new_board._board = copy.deepcopy(self._board)
        return new_board
    
    def __getitem__(self, i):
        '''
        Return ith row.
        '''
        return self._board[i]
    
    def __str__(self):        
        return "\n".join(str(row) for row in self._board)
    
    @staticmethod
    def _is_diag(x1, y1, x2, y2):
        return abs(x1-x2) == abs(y1-y2)
    
    def place(self, row, col):
        '''
        Place queen at (row, col). Mark all cells that can be attacked on the board as Block.
        '''
        assert self[row][col] == POS.Empty, f"Can not place in {(row, col)} its {self[row][col]}"
        self[row][col] = POS.Place

        #block row, col, and diagnoals
        for (i, j) in itertools.product(range(self.size()), range(self.size())):
            if i == row and j == col:
                continue        
            # for all cells along row, col and diagonally
            if i == row or j == col or Board._is_diag(row, col, i, j):
                self[i][j] = POS.Block
                    
    def __repr__(self):
        return str(self)
    
    def size(self):
        return len(self._board)
    
    def get_empty_cols(self, row):
        '''
        Return columns that are Empty in a row
        '''
        return [col for col in range(self.size()) if self._board[row][col] == POS.Empty]

function_call_counter = 0    
def solve(board, current_row):
    global function_call_counter
    function_call_counter += 1
    
    if current_row == board.size():         # done placing all queens.
        print(f"Found solution [{function_call_counter}]: ")
        print(board)
        return
    
    empty_cols = board.get_empty_cols(current_row)  
    if not empty_cols:
        #No more empty cols in row - backtrack
        return 

    # try each available empty cell to see if wwe can find a solution.
    for col in empty_cols:
        # make a copy, so that if do backtrack, we still have the previous state of the board.
        next_board = board.copy()
        #placing at current_row, col, and update the board
        next_board.place(current_row, col)
        # search and find solutions for remaining board.
        solve(next_board, current_row+1) 

            
solve(Board(8), 0)


Found solution [114]: 
[Q, X, X, X, X, X, X, X]
[X, X, X, X, Q, X, X, X]
[X, X, X, X, X, X, X, Q]
[X, X, X, X, X, Q, X, X]
[X, X, Q, X, X, X, X, X]
[X, X, X, X, X, X, Q, X]
[X, Q, X, X, X, X, X, X]
[X, X, X, Q, X, X, X, X]
Found solution [148]: 
[Q, X, X, X, X, X, X, X]
[X, X, X, X, X, Q, X, X]
[X, X, X, X, X, X, X, Q]
[X, X, Q, X, X, X, X, X]
[X, X, X, X, X, X, Q, X]
[X, X, X, Q, X, X, X, X]
[X, Q, X, X, X, X, X, X]
[X, X, X, X, Q, X, X, X]
Found solution [174]: 
[Q, X, X, X, X, X, X, X]
[X, X, X, X, X, X, Q, X]
[X, X, X, Q, X, X, X, X]
[X, X, X, X, X, Q, X, X]
[X, X, X, X, X, X, X, Q]
[X, Q, X, X, X, X, X, X]
[X, X, X, X, Q, X, X, X]
[X, X, Q, X, X, X, X, X]
Found solution [192]: 
[Q, X, X, X, X, X, X, X]
[X, X, X, X, X, X, Q, X]
[X, X, X, X, Q, X, X, X]
[X, X, X, X, X, X, X, Q]
[X, Q, X, X, X, X, X, X]
[X, X, X, Q, X, X, X, X]
[X, X, X, X, X, Q, X, X]
[X, X, Q, X, X, X, X, X]
Found solution [256]: 
[X, Q, X, X, X, X, X, X]
[X, X, X, Q, X, X, X, X]
[X, X, X, X, X, Q, X, X]
[X, X, X, 

## Search based on Permuations with Backtracking

We use permutations to represent a solution, but instead build the permutation incrementally.  When we find invalid permutation (i.e diag attack), we backtrack by removing the last col position added to the permutation and try a next possible value.


In [9]:
def generate_solutions(perm, n):
    
    if len(perm) == n:
        # its a solution
        print(perm)
        return
    
    for k in range(n):
        if k not in perm:
            # try next possible untested col positions 
            perm.append(k)
            if is_valid(perm):
                generate_solutions(perm, n)
            perm.pop()
            
def is_valid(perm):
    # check if the last position added keeps the permutation still valid
    i = len(perm) - 1
    # diag check
    for j in range(i):
        if (i - j) == abs(perm[i] - perm[j]):
            return False
    return True


generate_solutions([], 8)     
            

[0, 4, 7, 5, 2, 6, 1, 3]
[0, 5, 7, 2, 6, 3, 1, 4]
[0, 6, 3, 5, 7, 1, 4, 2]
[0, 6, 4, 7, 1, 3, 5, 2]
[1, 3, 5, 7, 2, 0, 6, 4]
[1, 4, 6, 0, 2, 7, 5, 3]
[1, 4, 6, 3, 0, 7, 5, 2]
[1, 5, 0, 6, 3, 7, 2, 4]
[1, 5, 7, 2, 0, 3, 6, 4]
[1, 6, 2, 5, 7, 4, 0, 3]
[1, 6, 4, 7, 0, 3, 5, 2]
[1, 7, 5, 0, 2, 4, 6, 3]
[2, 0, 6, 4, 7, 1, 3, 5]
[2, 4, 1, 7, 0, 6, 3, 5]
[2, 4, 1, 7, 5, 3, 6, 0]
[2, 4, 6, 0, 3, 1, 7, 5]
[2, 4, 7, 3, 0, 6, 1, 5]
[2, 5, 1, 4, 7, 0, 6, 3]
[2, 5, 1, 6, 0, 3, 7, 4]
[2, 5, 1, 6, 4, 0, 7, 3]
[2, 5, 3, 0, 7, 4, 6, 1]
[2, 5, 3, 1, 7, 4, 6, 0]
[2, 5, 7, 0, 3, 6, 4, 1]
[2, 5, 7, 0, 4, 6, 1, 3]
[2, 5, 7, 1, 3, 0, 6, 4]
[2, 6, 1, 7, 4, 0, 3, 5]
[2, 6, 1, 7, 5, 3, 0, 4]
[2, 7, 3, 6, 0, 5, 1, 4]
[3, 0, 4, 7, 1, 6, 2, 5]
[3, 0, 4, 7, 5, 2, 6, 1]
[3, 1, 4, 7, 5, 0, 2, 6]
[3, 1, 6, 2, 5, 7, 0, 4]
[3, 1, 6, 2, 5, 7, 4, 0]
[3, 1, 6, 4, 0, 7, 5, 2]
[3, 1, 7, 4, 6, 0, 2, 5]
[3, 1, 7, 5, 0, 2, 4, 6]
[3, 5, 0, 4, 1, 7, 2, 6]
[3, 5, 7, 1, 6, 0, 2, 4]
[3, 5, 7, 2, 0, 6, 4, 1]
[3, 6, 0, 7, 4, 1, 5, 2]
